In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [4]:
%%capture pwd
!pwd

In [5]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('three-centuries-of-uk-business-cycles-what-lessons-for-today', 300, 500)

# Fig 1

In [6]:
df = pd.read_excel("raw/Figures.xlsx", sheet_name='Fig1A')
df.columns=['year','gdp','gva','lngdp','lngva']

In [7]:
f = "fig1_gdp"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,gdp,gva,lngdp,lngva
0,1700,10782.766444,9179.795776,9.285704,9.124760
1,1701,11753.215424,10005.977395,9.371882,9.210938
2,1702,11451.297963,9748.943114,9.345858,9.184914
3,1703,10718.069845,9124.717001,9.279686,9.118742
4,1704,12756.012703,10859.698403,9.453758,9.292814


In [8]:
df = pd.read_excel("raw/Figures.xlsx", sheet_name='Fig1B')
df.columns=['peak','trough']

In [9]:
f = "fig1b_recess"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,peak,trough
0,1701,1703
1,1704,1706
2,1708,1710
3,1712,1713
4,1714,1715


In [10]:
# from urllib import request
# with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/en-GB.json') as f:
#     de_format = json.load(f)
# with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/en-GB.json') as f:
#     de_time_format = json.load(f)
# alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

In [113]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            format='.0f'
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "g:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            format='~s',
            labelExpr= "((datum.label=='2T')?'£2 trillion':replace(replace(datum.label,'G',' billion'),'T',' trillion'))"
        ),
        scale=alt.Scale(type='log',domain=[5000000000,2000000000000],nice=False),
    )
).transform_calculate(g='datum.gdp*1000000')
line2 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "v:Q",
    )
).transform_calculate(v='datum.gva*1000000')
# points = line.mark_point(
#     size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
# )
# area = base.mark_area(opacity=0.4, color=colors["eco-turquiose"]).encode(
#     y="e1:Q", y2="e2:Q"
# )
labels = (
    alt.Chart(pd.DataFrame([{"x": "2009", "y": 1900000000000,"t":'GDP at market prices'}, 
                            {"x": "2009", "y": 1400000000000,"t":'GVA at basic prices'}]))
    .mark_text(align='left',dx=5)
    .encode(x=alt.X("x:Q", sort=[]), y="y:Q",text='t:N',color=alt.Color('t:N',legend=None,scale=alt.Scale(range=[colors['eco-turquiose'],
                                                                                                                 colors['eco-dot']])))
)
rulers = alt.Chart(f1b).encode(
    x=alt.X(
        "peak:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            format='.0f'
        ),
        scale=alt.Scale(domain=[1700,2020],nice=False)
    ),
    x2='trough:Q'
).mark_rect(color='gray',opacity=0.2)
layer1 = (
    ((rulers+line+line2+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")#.configure(customFormatTypes=True)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [115]:
df = pd.read_excel("raw/Figures.xlsx", sheet_name='Fig2')

In [116]:
f = "fig2_pass"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,h,Average,Lower 95,Upper 95
0,0,0.000000,0.000000,0.000000
1,1,-2.456968,-3.043147,-1.870789
2,2,-1.489501,-2.905568,-0.073433
3,3,0.510392,-0.966366,1.987150
4,4,1.983706,0.380320,3.587092


In [162]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "h:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="years from peak ➡",
            titleY=-15,
            titleX=157,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=4,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
    )
)
line = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y(
        "Average:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Deviation from peak (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
band = (
    base.mark_area(opacity=0.4, color=colors["eco-light-blue"]).encode(
    y="Lower 95:Q", y2="Upper 95:Q")
)
area = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": 0,"y2": 6}, {"x": 5, "y": 0,'y2':6}]))
    .mark_area(opacity=0.1, color=colors["eco-turquiose"]).encode(
    y="y:Q", y2="y2:Q",x='x:Q')
)
area2 = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": 0,"y2": -4}, {"x": 5, "y": 0,'y2':-4}]))
    .mark_area(opacity=0.1, color=colors["eco-dot"]).encode(
    y="y:Q", y2="y2:Q",x='x:Q')
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x":0, "y": 0}, {"x": 5, "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:Q", sort=[]), y="y:Q")
)
# axis2 = (
#     alt.Chart(pd.DataFrame([{"x": "75", "y": -0.16}, {"x": "75", "y": 0.05}]))
#     .mark_line(strokeWidth=1, xOffset=25, color=colors["eco-dot"])
#     .encode(x=alt.X("x:N", sort=[]), y="y:Q")
# )
layer1 = (
    ((area+area2+band+axis1+line).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [176]:
df = pd.read_excel("raw/NIESR_Chart_-_Figure_3_for_Econ_Observatory.xlsx", nrows=21,sheet_name='Data',usecols='I:Q')
df=df.drop(['Unnamed: 14','2019Q3/Q4','1920Q2/3'],axis=1).stack().reset_index()
df.columns=['dt','quarter','value']

In [177]:
f = "fig3_mpc"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,dt,quarter,value
0,0,1930Q1,0.0
1,0,1973Q2,0.0
2,0,1979Q2,0.0
3,0,1990Q2,0.0
4,0,2008Q1,0.0


In [239]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "dt:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="quarters from peak ➡",
            titleY=-15,
            titleX=277,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=4,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
    ),
    color=alt.Color('quarter:N',legend=None)
)
line = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Deviation from peak (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
area = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": 0,"y2": 6}, {"x": 20, "y": 0,'y2':6}]))
    .mark_area(opacity=0.1, color=colors["eco-turquiose"]).encode(
    y="y:Q", y2="y2:Q",x='x:Q')
)
labels=line.mark_text(align='left',dx=5).encode(text='quarter:N').transform_filter('datum.dt==20')
labels1=labels.transform_filter('datum.quarter!="1973Q2"')
labels2=labels.mark_text(align='left',dx=5,dy=12).transform_filter('datum.quarter=="1973Q2"')
line2=line.mark_line(strokeWidth=3,stroke=colors['eco-light-blue']).transform_filter('datum.quarter=="2022Q2"')
labels3=(
    alt.Chart(pd.DataFrame([{"x": 13.5, "y": -3.4,"t": '2022Q2 MPR Forecast'},
                           {"x": 13.5
                            , "y": -4.1,"t": 'Market Interest Rates'}]))
    .mark_text(align='left',dx=5,color=colors['eco-light-blue'],fontWeight='bold').encode(text='t:N',x='x:Q',y='y:Q')
)
area2 = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": 0,"y2": -7}, {"x": 20, "y": 0,'y2':-7}]))
    .mark_area(opacity=0.1, color=colors["eco-dot"]).encode(
    y="y:Q", y2="y2:Q",x='x:Q')
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x":0, "y": 0}, {"x": 20, "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:Q", sort=[]), y="y:Q")
)
# axis2 = (
#     alt.Chart(pd.DataFrame([{"x": "75", "y": -0.16}, {"x": "75", "y": 0.05}]))
#     .mark_line(strokeWidth=1, xOffset=25, color=colors["eco-dot"])
#     .encode(x=alt.X("x:N", sort=[]), y="y:Q")
# )
layer1 = (
    ((area+area2+axis1+line+labels1+labels2+line2+labels3).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)